In [1]:
import cv2
import numpy as np
import os

In [2]:
# Load YOLO model files
yolo_net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
layer_names = yolo_net.getLayerNames()
output_layers = [layer_names[i - 1] for i in yolo_net.getUnconnectedOutLayers()]

error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\dnn\src\darknet\darknet_importer.cpp:210: error: (-212:Parsing error) Failed to open NetParameter file: yolov3.cfg in function 'cv::dnn::dnn4_v20241223::readNetFromDarknet'


In [ ]:
# Load the class names (COCO dataset contains a 'dog' class)
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

In [ ]:
# Function to detect dog in an image
def detect_and_crop_dog(image_path, output_path):
    # Load image
    img = cv2.imread(image_path)
    height, width, channels = img.shape

    # Prepare the image for YOLO
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    yolo_net.setInput(blob)
    outputs = yolo_net.forward(output_layers)

    # Process YOLO outputs
    for out in outputs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            
            # If dog (class_id = 16 in COCO dataset) is detected with high confidence
            if confidence > 0.5 and classes[class_id] == "dog":
                # Get the bounding box coordinates
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates for cropping
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                # Crop the image around the dog
                cropped_img = img[y:y + h, x:x + w]

                # Save the cropped image
                cv2.imwrite(output_path, cropped_img)
                print(f"Cropped image saved at {output_path}")
                return

    print(f"No dog detected in {image_path}.")

In [ ]:
# Process all images in the './data/train/ginger/' directory
input_dir = os.path.abspath('./data/train/ginger/')
output_dir = os.path.abspath('./data/train/ginger/cropped/')

# Create output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
# Loop through all files in the directory and apply the cropping function
for index, filename in enumerate(os.listdir(input_dir)):
    if filename.endswith(".jpg"):
        image_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, f"ginger{index}.jpg")
        
        # Apply the cropping function
        detect_and_crop_dog(image_path, output_path)

print("All images processed.")